In [1]:
pip install langchain

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install langchain-pinecone

Note: you may need to restart the kernel to use updated packages.


In [3]:
!pip install PyPDF

In [4]:
pip install langchain-openai 

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install tiktoken

Note: you may need to restart the kernel to use updated packages.


In [6]:
from langchain.document_loaders import PyPDFDirectoryLoader 
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddingsS
from langchain.llms import openai
from langchain.vectorstores import Pinecone
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
import os

In [7]:
!mkdir PDFs

A subdirectory or file PDFs already exists.


In [8]:
loader=PyPDFDirectoryLoader("PDFs")

In [9]:
data=loader.load()

In [10]:
data[0]

Document(page_content='You Only Look Once:\nUniﬁed, Real-Time Object Detection\nJoseph Redmon∗, Santosh Divvala∗†, Ross Girshick¶, Ali Farhadi∗†\nUniversity of Washington∗, Allen Institute for AI†, Facebook AI Research¶\nhttp://pjreddie.com/yolo/\nAbstract\nWe present YOLO, a new approach to object detection.\nPrior work on object detection repurposes classiﬁers to per-\nform detection. Instead, we frame object detection as a re-\ngression problem to spatially separated bounding boxes and\nassociated class probabilities. A single neural network pre-\ndicts bounding boxes and class probabilities directly from\nfull images in one evaluation. Since the whole detection\npipeline is a single network, it can be optimized end-to-end\ndirectly on detection performance.\nOur uniﬁed architecture is extremely fast. Our base\nYOLO model processes images in real-time at 45 frames\nper second. A smaller version of the network, Fast YOLO,\nprocesses an astounding 155 frames per second while\nstill ac

In [11]:
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=150,
    chunk_overlap=20,
    length_function=len,
    is_separator_regex=False,
)

In [12]:
test_chunks = text_splitter.split_documents(data)

In [13]:
test_chunks

[Document(page_content='You Only Look Once:\nUniﬁed, Real-Time Object Detection\nJoseph Redmon∗, Santosh Divvala∗†, Ross Girshick¶, Ali Farhadi∗†', metadata={'source': 'PDFs\\yolo.pdf', 'page': 0}),
 Document(page_content='University of Washington∗, Allen Institute for AI†, Facebook AI Research¶\nhttp://pjreddie.com/yolo/\nAbstract', metadata={'source': 'PDFs\\yolo.pdf', 'page': 0}),
 Document(page_content='Abstract\nWe present YOLO, a new approach to object detection.\nPrior work on object detection repurposes classiﬁers to per-', metadata={'source': 'PDFs\\yolo.pdf', 'page': 0}),
 Document(page_content='form detection. Instead, we frame object detection as a re-\ngression problem to spatially separated bounding boxes and', metadata={'source': 'PDFs\\yolo.pdf', 'page': 0}),
 Document(page_content='associated class probabilities. A single neural network pre-\ndicts bounding boxes and class probabilities directly from', metadata={'source': 'PDFs\\yolo.pdf', 'page': 0}),
 Document(page_c

In [14]:
print(test_chunks[0].page_content)

You Only Look Once:
Uniﬁed, Real-Time Object Detection
Joseph Redmon∗, Santosh Divvala∗†, Ross Girshick¶, Ali Farhadi∗†


In [15]:
print(test_chunks[1].page_content)

University of Washington∗, Allen Institute for AI†, Facebook AI Research¶
http://pjreddie.com/yolo/
Abstract


In [16]:
len(test_chunks)

357

In [21]:
from dotenv import load_dotenv
load_dotenv()

True

In [22]:
embeddings=OpenAIEmbeddings(api_key=os.environ['OPENAI_API_KEY'])

C:\Users\ajays\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [23]:
embeddings

OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x000001A22F5F0A90>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x000001A22F5F8DD0>, model='text-embedding-ada-002', deployment='text-embedding-ada-002', openai_api_version='', openai_api_base=None, openai_api_type='', openai_proxy='', embedding_ctx_length=8191, openai_api_key='sk-proj-8y3pt4Q1Mlu73Ty2KL4GT3BlbkFJaGhcDQ5aEZpJxl4ahITJ', openai_organization=None, allowed_special=set(), disallowed_special='all', chunk_size=1000, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None)

In [24]:
!pip install pinecone-client


In [40]:
from pinecone import Pinecone, ServerlessSpec

In [26]:
pc = Pinecone(api_key=os.environ['PINECONE_API_KEY'])

In [27]:
index_name = "docs-quickstart-index"

if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=1536,
        metric="cosine",
        spec=ServerlessSpec(
            cloud='aws', 
            region='us-east-1'
        ) 
    ) 


In [28]:
from langchain_pinecone import PineconeVectorStore

In [29]:
vectorstore = PineconeVectorStore(index_name=index_name, embedding=embeddings)
docsearch=vectorstore.add_documents(test_chunks)

In [30]:
retriever = vectorstore.as_retriever(search_kwargs={'k':1})

In [31]:
retriever

VectorStoreRetriever(tags=['PineconeVectorStore', 'OpenAIEmbeddings'], vectorstore=<langchain_pinecone.vectorstores.PineconeVectorStore object at 0x000001A22FF80990>, search_kwargs={'k': 1})

In [32]:
from langchain_openai import ChatOpenAI

In [33]:
query = "yolo outperforms which model?"
docs=vectorstore.similarity_search(query)

In [34]:
docs

[Document(page_content='train YOLO achieves higher performance.', metadata={'page': 6.0, 'source': 'PDFs\\yolo.pdf'}),
 Document(page_content='YOLO is extremely fast at test', metadata={'page': 3.0, 'source': 'PDFs\\yolo.pdf'}),
 Document(page_content='work, YOLO outperforms top detection methods like', metadata={'page': 1.0, 'source': 'PDFs\\yolo.pdf'}),
 Document(page_content='and YOLO pushes the state-of-the-art in', metadata={'page': 7.0, 'source': 'PDFs\\yolo.pdf'})]

In [35]:
llm = ChatOpenAI()

In [36]:
qa = RetrievalQA.from_chain_type(llm,
                                          chain_type='stuff',
                                          retriever=vectorstore.as_retriever(),
                                         
                                          )

In [43]:
query = "what is the accuracy of yolo?"

In [44]:
qa.run(query)

'The accuracy of YOLO can vary depending on the specific implementation and dataset used for training. Generally, YOLO is known to be a fast and accurate object detector, but the exact accuracy percentage would depend on the specific factors mentioned above.'